# Learning with Keras 
----------------------------------------------------
This is a demo of a Binary classification, I apply all encoding techniches que you already know, we are no looking to do a bag of words instead we are trying to figureout the sequence of words, order matter in a sentences and in particular in Sentiment analysis

# Loading tools 

In [ ]:
#this is to force to be compatible con Python 3.xx
#from __future__ import absolute_import
#from __future__ import print_function
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
 
import seaborn as sns
import nltk
import csv
import itertools
import operator
import unicodedata
import nltk
import sys
import os
import time, re
from datetime import datetime
import collections
from bs4 import BeautifulSoup 
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
stemmer = SnowballStemmer("english")


## Loading utilities tools to process text

In [ ]:
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.cross_validation import train_test_split

## Loading actual Tools and Layers of Deep Learning

In [ ]:

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization #https://github.com/fchollet/keras/blob/master/examples/kaggle_otto_nn.py
from keras.layers.advanced_activations import PReLU, LeakyReLU, ELU, ParametricSoftplus, ThresholdedLinear, ThresholdedReLU
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.optimizers import SGD
from keras.callbacks import History, EarlyStopping
from keras.preprocessing.text import Tokenizer

# this is to be sure that al results in development are comparable
np.random.seed(1337)  # for reproducibility

## actual code

In [ ]:
df = pd.read_csv("data/Tweets_airline.csv")
vocabulary_size = 0
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

#print (df.shape)
#print (df.info())

## Some small utilities

In [ ]:
def cleanword(w):        
    return re.sub('[^a-zA-Z0-9,]' , ' ' , w)  

def cleantext(review):
    review = BeautifulSoup(review ,"lxml").get_text()
    review_words = cleanword(review.lower()).split()    
#    stop = stopwords.words('english')
#    stemmed_words = [stemmer.stem(w) for w in review_words if w not in stop]
#    return " ".join(stemmed_words)
    return " ".join(review_words)


df["text"]  = df["text"].apply(cleantext)
print "An example of a single message:", df.head(5)['text']


## Tokenize all sentences
it is a good Idea to tokenize with Keras, instead of bag of words


In [ ]:
################# ML ##############################################3

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split 

X_clean = df["text"] 

sentences = X_clean
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d uniq words tokens." % len(word_freq.items())
w_tokens = len(word_freq.items())
vocabulary_size = w_tokens +1
print "Example of a tokenize sentence", tokenized_sentences[33]

In [ ]:
max_words= len(word_freq.items())+1
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([w,i] for i,w in enumerate(index_to_word))
print "using vocabulary size %d" % vocabulary_size
for i, sent in enumerate (tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
X_clean = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])
X_clean = np.asarray(X_clean)
print "Example of a sequence sentence", X_clean[33]


In [ ]:
# LabelEncoder is a utility class to help normalize labels such 
# that they contain only values between 0 and n_classes-1. 
le = preprocessing.LabelEncoder()
le.fit(df["airline_sentiment"])
print "Example of a sentiment sentence: ", df["airline_sentiment"][33]
y = le.transform(df["airline_sentiment"])
y = np.asarray(y)
print "Example of a sentiment After Encoder: ",y[33]

X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2) #, random_state=42

print("Convert class vector to binary class matrix (for use with categorical_crossentropy)")
y_train = np.array(y_train, dtype = int)
y_test = np.array(y_test, dtype = int)
nb_classes = np.max(y_train)+1
nb_classes = int(nb_classes)
print(nb_classes, 'classes')

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print "Example of a sentiment After categorical Encoder: ",Y_train[1]

In [ ]:
print("Vectorizing sequence data...")
tokenizer = Tokenizer(nb_words=max_words)
X_train = tokenizer.sequences_to_matrix(X_train, mode="binary")
X_test = tokenizer.sequences_to_matrix(X_test, mode="binary")

#X_train = tokenizer.sequences_to_matrix(X_train, mode="count")
#X_test = tokenizer.sequences_to_matrix(X_test, mode="count")
print X_train[1]

## Some parameters 
That make the model perform better or worse depending on the selection

In [ ]:
max_features = 350
maxlen = 350  # cut texts after this number of words (among top max_features most common words)
batch_size = 64
max_words = w_tokens+1 # you should run it for the first time anc copy the value of mismatch dimension [1] (32,15101)x(11501,64)->(32,64)
nb_epoch = 300
value_dense = 64
drop_layer = 0.5
##
# BatchNormalization
# Normalize the activations of the previous layer at each batch, 
# i.e. applies a transformation that maintains the mean activation 
# close to 0 and the activation standard deviation close to 1.
#
##
b_norm = 1
extra_layers = 5

![alt_text](http://scs.ryerson.ca/~aharley/neural-networks/images/fcn_visualization7_big.png "_")

## Building the Actual Model 

In [ ]:
print("Building model...")
model = Sequential()
model.add(Dense(value_dense, input_shape=(max_words,)))
model.add(Activation('relu'))
if(b_norm):
    model.add(BatchNormalization())
model.add(Dropout(drop_layer))
    
for i in range(extra_layers):
    model.add(Dense((value_dense), input_shape=(max_words,)))
    model.add(Activation('relu'))
    if(b_norm):
        model.add(BatchNormalization())
    model.add(Dropout((drop_layer)))
    
model.add(Dense(nb_classes))
model.add(Activation('softmax')) #softmax
# this is to use Stochastic Gradiend Decendant
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='Adadelta') #Adadelta outperform SGD

![alt text](http://scs.ryerson.ca/~aharley/neural-networks/images/one_target_graddescent.gif "http://scs.ryerson.ca/~aharley/neural-networks/")

## Generate image of the model that we are using 

In [ ]:
from keras.utils.visualize_util import plot
plot(model, to_file='model.png')
from IPython.display import SVG
from keras.utils.visualize_util import to_graph
SVG(to_graph(model).create(prog='dot', format='svg'))

In [ ]:
history = History()
early_stopping = EarlyStopping(patience=5, verbose=1)
model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, 
          show_accuracy=True, validation_split=0.1, callbacks=[history,early_stopping])
score = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=1, show_accuracy=True)
print('Test score:', score[0])
print('Test accuracy: %.2f' % (score[1]*100),"%")
#print "history.history",history.totals
classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)
pred = model.predict(X_test)

## Evaluate your Results